In [ ]:
import fastf1
import pandas as pd

In [ ]:
YEARS = [2024] # fix to include 2022 ground effect

acc = []

for year in YEARS:
  events = fastf1.events.get_event_schedule(year)

  for i, event in events.iterrows():
    if event.is_testing():
      continue

    session = fastf1.get_session(year, event['RoundNumber'], 'Race')
    session.load(telemetry=True)

    for driver in session.drivers:
      try:
        df = session.laps.pick_drivers(driver).get_pos_data().add_driver_ahead()

        df['DriverId'] = driver
        df['DriverName'] = session.get_driver(driver).FullName
        df['RoundNumber'] = event['RoundNumber']
        df['Country'] = event['Country']
        df['Location'] = event['Location']
        df['OfficialEventName'] = event['OfficialEventName']
        # df['EventDate'] = event['EventDate']
        df['EventName'] = event['EventName']
        df['EventFormat'] = event['EventFormat']
        
        acc.append(df)
      except Exception as e:
        pass

df = pd.concat(acc, ignore_index=True)
df


In [ ]:
dff = df[df['DistanceToDriverAhead'] < 5]
dff

In [ ]:
import sys

dff.to_parquet('output.parquet', index=False)

with open('output.parquet', 'rb') as f:
    sys.stdout.buffer.write(f.read())


In [ ]:
import pandas as pd
import numpy as np

# Create a sample DataFrame with DistanceToDriverAhead column
data = {
    'Time': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'DriverAhead': ['Hamilton', 'Hamilton', 'Verstappen', 'Verstappen', None, 
                   'Leclerc', 'Leclerc', None, 'Sainz', 'Sainz'],
    'DistanceToDriverAhead': [300, 250, 600, 550, np.nan, 400, 350, np.nan, 700, 650],
    'OtherData': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
}
df = pd.DataFrame(data)

# Display the original DataFrame
# print("Original DataFrame:")
# print(df)
# print("\n")

# Create a mask with all our conditions:
# 1. DriverAhead changes from previous row
# 2. Current row has a driver ahead (not NaN)
# 3. Previous row had a driver ahead (not NaN)
# 4. Previous row had DistanceToDriverAhead < 500
mask = (df['DriverAhead'].shift() != df['DriverAhead']) & \
       (df['DriverAhead'].notna()) & \
       (df['DriverAhead'].shift().notna()) & \
       (df['DistanceToDriverAhead'].shift() < 500)

# Include the first row if it has a driver ahead
first_row_mask = df.index == 0
if len(df) > 0 and pd.notna(df.loc[0, 'DriverAhead']):
    mask = mask | first_row_mask

# Filter the DataFrame
filtered_df = df[mask]
filtered_df = filtered_df.reset_index(drop=True)

# Display the filtered DataFrame
# print("Filtered DataFrame (only qualifying driver-to-driver transitions):")
# print(filtered_df)